In [1]:
import json
# from statistics import median_grouped
import pandas as pd
# from pyrsistent import v
import mne
import os
import time
import tqdm
import math
import glob

In [2]:
#
df = pd.read_excel('总表5.xlsx',engine='openpyxl')

# Rename columns to match the old format
df.rename(columns={'视频标题': 'item_id', '事件类型': 'event', '时间戳': 'local_time_ms'}, inplace=True)

# Replace values in 'event' column
df['event'] = df['event'].replace({
    '开始': 'video_play',
     '结束': 'video_end',
     '暂停': 'pause',
    '下一个': 'nextone',
    '上一个':'lastone'
})

# Replace any event containing '评分' with 'valence_arousal'
df['event'] = df['event'].apply(lambda x: 'valence_arousal' if '评分' in x else x)

In [ ]:
'''
'LAB1-huqifan','LAB1-cangyueyang','LAB1-hongyurui','LAB1-fanhao','LAB1-dongyimeng','LAB1-houlinzhi','LAB1-jiwenjun','LAB1-lujianing','LAB1-miaoshengze',
             'LAB1-wanfangwei','LAB1-wangxiaoting','LAB1-wangzhengni','LAB1-yangchen','LAB1-zhangxue','LAB1-liangqihang','LAB1-daisiwei',
             'LAB1-zhangyutong','LAB1-mengfanjie','LAB1-zhangchenxi','LAB1-liangyanshu'
'LAB2-huqifan','LAB2-cangyueyang','LAB2-hongyurui','LAB2-fanhao','LAB2-dongyimeng','LAB2-houlinzhi','LAB2-jiwenjun','LAB2-lujianing','LAB2-miaoshengze',
             'LAB2-wanfangwei','LAB2-wangxiaoting','LAB2-wangzhengni','LAB2-yangchen','LAB2-zhangxue','LAB2-liangqihang','LAB2-daisiwei',
             'LAB2-zhangyutong','LAB2-mengfanjie','LAB2-zhangchenxi','LAB2-liangyanshu'
'''

In [ ]:
# Adding a new column 'like' based on 'item_id' values
def determine_like(item_id):
    if pd.isna(item_id):
        return None  # or a default value of your choice
    if '正面' in item_id:
        return 0
    elif '负面' in item_id:
        return 1
    elif 'neutral' in item_id:
        return 2
    elif 'sad' in item_id:
        return 3
    elif 'fear' in item_id:
        return 4
    elif 'happy' in item_id:
        return 5
    elif 'END' in item_id:
        return 6
    else:
        return None  # or a default value of your choice

In [ ]:
df = df.dropna(subset=['item_id'])
df['video_type'] = df['item_id'].apply(determine_like)

In [ ]:
#拆分excel
def split_and_save_excel(df, column_name, folder_path):
    # 确保文件夹路径存在
    os.makedirs(folder_path, exist_ok=True)

    # 获取独特的会员账号
    unique_values = df[column_name].unique()

    # 对每个独特的会员账号进行遍历
    for value in unique_values:
        # 为当前会员账号创建一个新的数据框
        df_subset = df[df[column_name] == value]

        # 构建保存文件的路径
        file_path = os.path.join(folder_path, f'{value}.xlsx')

        # 保存数据框为Excel文件
        df_subset.to_excel(file_path, index=False)
# 调用函数
split_and_save_excel(df, '会员账号', r"D:\SIGIR_EEG\拆分的log5")

In [ ]:
def dic_v2k(trigger2event_dict):
    re = {}
    for k,v in trigger2event_dict.items():
        re[v] = k
    return re

In [ ]:
def load_txt(file_name):
    trigger2trigger2time = {}
    with open(file_name) as f:
        for line in f.readlines():
            t1, t2, time = line.strip().split(':')
            if t1 not in trigger2trigger2time.keys():
                trigger2trigger2time[t1] = {}
            trigger2trigger2time[t1][t2] = time
    return trigger2trigger2time

In [ ]:
def timestamp_convert_localdate(timestamp,time_format="%Y/%m/%d %H:%M:%S"):
    # 按照当前设备时区来进行转换，比如当前北京时间UTC+8
    timeArray = time.localtime(timestamp)
    styleTime = time.strftime(str(time_format), timeArray)
    return styleTime

In [ ]:
def filter_255(events_from_annot):
    re = []
    for item in events_from_annot:
        if item[-1] != 255:
            re.append(item)
    return re

In [ ]:
'''
def load_excel(file_name):

    df = pd.read_excel(file_name)
    df.sort_values("local_time_ms", inplace=True, ignore_index=True)

    v2info = {}
    for i in range(len(df)):
        current_id = df.loc[i]['item_id']

        if current_id not in v2info.keys():
            v2info[current_id] = {'video_type': int(0), 'start_time': -1, 'end_time': -1}

        # Updating the 'like' handling to range from 0 to 5
        current_like = df.loc[i]['video_type']
        if 0 <= current_like <= 5:
            v2info[df.loc[i]['item_id']]['video_type'] = int(current_like)

        # Update 'start_time' for every 'video_play' event
        if df.loc[i]['event'] == 'video_play':
            v2info[current_id]['start_time'] = df.loc[i]['local_time_ms']

        # Update 'end_time' for every 'video_end' or 'nextone' event
        elif df.loc[i]['event'] in ['video_end', 'nextone']:
            v2info[current_id]['end_time'] = df.loc[i]['local_time_ms']

    return v2info

# The code has been updated to update 'start_time' for every 'video_play' event,
# and 'end_time' for every 'video_end' or 'nextone' event.
'''

In [ ]:
#v2info=load_excel('./lab1-log/'+'LAB1-cangyueyang'+'.xlsx')

In [ ]:
#【提取被试的评分信息】
def load_data_from_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return json.load(file)
user_data = load_data_from_file('user_data_full.json')

In [ ]:
pinyin_map = {
 '洪宇睿': 'hongyurui',
 '樊昊': 'fanhao',
 '梁起航': 'liangqihang',
 '梁雁舒': 'liangyanshu',
 '卢佳宁': 'lujianing',
 '张雨桐': 'zhangyutong',
 '万方伟': 'wanfangwei',
 '张晨曦': 'zhangchenxi',
 '苍岳洋': 'cangyueyang',
 '王晓婷': 'wangxiaoting',
 '胡启帆': 'huqifan',
 '孟凡杰': 'mengfanjie',
 '戴思巍': 'daisiwei',
 '王峥妮': 'wangzhengni',
 '张雪': 'zhangxue',
 '苗圣泽': 'miaoshengze',
 '侯林之': 'houlinzhi',
 '董艺萌': 'dongyimeng',
 '吉文俊': 'jiwenjun',
 '杨晨': 'yangchen',
 '赵晨淞':'zhaochensong',
 '陈蓉':'chenrong',
 '郭弘扬':'guohongyang',
 '陈星宇':'chenxingyu'
 }
#命名人物列表（按顺序）
renwu=['爱迪生','曹操','哥伦布','汉武帝','叶卡捷琳娜','老白','迈克','牛顿','秦始皇','项羽']
# Function to load Excel and add 'tend' information based on the character list 'renwu' and user_data
def load_excel_with_tend(file_name, user_data, renwu, pinyin_map):
    df = pd.read_excel(file_name)
    df.sort_values("local_time_ms", inplace=True, ignore_index=True)

    char_to_index_map = {char: i for i, char in enumerate(renwu)}

    v2info = {}
    for i in range(len(df)):
        current_id = str(df.loc[i]['item_id'])  # 转换为字符串，以防止 JSON 序列化问题

        if current_id not in v2info:
            v2info[current_id] = {'video_type': 0, 'start_time': -1, 'end_time': -1, 'prev_start_time': -1, 'tend': 2}

        current_like = int(df.loc[i]['video_type'])  # 使用 int() 转换
        v2info[current_id]['video_type'] = current_like

        if df.loc[i]['event'] == 'video_play':
            v2info[current_id]['prev_start_time'] = v2info[current_id]['start_time']
            v2info[current_id]['start_time'] = int(df.loc[i]['local_time_ms'])  # 使用 int() 转换

        elif df.loc[i]['event'] in ['video_end', 'nextone']:
            v2info[current_id]['end_time'] = int(df.loc[i]['local_time_ms'])  # 使用 int() 转换
            v2info[current_id]['prev_start_time'] = -1

        if v2info[current_id]['start_time'] > v2info[current_id]['end_time'] and v2info[current_id]['prev_start_time'] != -1:
            v2info[current_id]['start_time'] = v2info[current_id]['prev_start_time']

        char_index = None
        for char in renwu:
            if char in current_id:
                char_index = char_to_index_map[char]
                break

        if char_index is not None:
            for name, pinyin in pinyin_map.items():
                if pinyin in str(df['会员账号'].values[0]):  # 转换为字符串
                    for this_user_data in user_data:
                        if this_user_data['name'] == name:
                            v2info[current_id]['tend'] = this_user_data['tend'][char_index]
                            break
                    break

    return v2info



In [ ]:
class Transformer:
    def __init__(self,):
        self.b = 0
        self.k = 0
    def fit(self, action, event):
        self.k = (event[1] - event[0]) / (action[1] - action[0])
        self.b = event[1] - action[1] * self.k
        return self.k, self.b
    def action2event(self,action):
        return int(self.k * action + self.b + 0.5)       

In [ ]:
def find_time(start_time, time2t2t, time_stamps, txt_info):
    if start_time < time_stamps[0] or start_time > time_stamps[-1]:
        return None
    for i, time_stamp in enumerate(time_stamps):
        # mising three triggers
        if time_stamps[i+1] - time_stamp > 10000 * 3:
            return None
        if start_time >= time_stamp and start_time < time_stamps[i+1]:
            real_time = [time_stamp, time_stamps[i+1]]
            eeg_time = []
            for rtime in real_time:
                t1, t2 = time2t2t[rtime]
                eeg_time.append(txt_info[t1][t2]['eeg_time'])
            transformer = Transformer()
            transformer.fit(real_time, eeg_time)
            if abs(transformer.k - 1000) > 50:
                with open('tmp.txt','a') as f:
                    f.write(str(real_time))
                    f.write('\t')
                    f.write(str(eeg_time))
                    f.write('\t')
                    f.write(str(t1)+'\t'+str(t2))
                    f.write('\n')
                print(transformer.k)
                return None
            return transformer.action2event(start_time)

In [ ]:
'''
def map_info(txt_info, events_from_annot, excel_info):
    
    event_group = []
    for i in range(len(events_from_annot)):
        if i + 1 < len(events_from_annot):
            time_diff = events_from_annot[i+1][0] - events_from_annot[i][0]
            if time_diff < 1050 and time_diff > 950:
                event_group.append([events_from_annot[i][2], events_from_annot[i+1][2], events_from_annot[i+1][0]])
                if str(events_from_annot[i][2]) in txt_info.keys() and str(events_from_annot[i+1][2]) in txt_info[str(events_from_annot[i][2])].keys():
                    print("txt:",txt_info[str(events_from_annot[i][2])][str(events_from_annot[i+1][2])])
                    print(str(events_from_annot[i][2]))
                    txt_info[str(events_from_annot[i][2])][str(events_from_annot[i+1][2])] = {
                        'time': float(txt_info[str(events_from_annot[i][2])][str(events_from_annot[i+1][2])]),
                        'eeg_time': float(events_from_annot[i+1][0])
                    }
    
    time2t2t = {}
    for t1 in txt_info.keys():
        for t2 in txt_info[t1].keys():
            if type(txt_info[t1][t2]) == dict:
                time2t2t[txt_info[t1][t2]['time']] = [t1, t2]
    time_stamps = list(sorted([float(item) for item in time2t2t.keys()]))

    v2info = excel_info
    for v in v2info.keys():
        start_time = int(v2info[v]['start_time'] / 1e3)
        end_time = int(v2info[v]['end_time'] / 1e3)
        time_diff = int(end_time - start_time)
        eeg_time = find_time(start_time, time2t2t, time_stamps, txt_info)
        if eeg_time != None:
            v2info[v]['eeg_start_time'] = int(eeg_time)
        eeg_time = find_time(end_time, time2t2t, time_stamps, txt_info)
        if eeg_time != None:
            v2info[v]['eeg_end_time'] = int(eeg_time)
    return v2info
'''

In [ ]:
#代码在处理不符合预期格式的数据时会更加健壮。如果 txt_info 中的值已经是一个字典，这段代码将不会尝试对它进行浮点数转换，从而避免了 TypeError。
def map_info(txt_info, events_from_annot, excel_info):
    
    event_group = []
    for i in range(len(events_from_annot)):
        if i + 1 < len(events_from_annot):
            time_diff = events_from_annot[i+1][0] - events_from_annot[i][0]
            if time_diff < 1050 and time_diff > 950:
                event_group.append([events_from_annot[i][2], events_from_annot[i+1][2], events_from_annot[i+1][0]])
                
                event_key = str(events_from_annot[i][2])
                next_event_key = str(events_from_annot[i+1][2])

                if event_key in txt_info and next_event_key in txt_info[event_key]:
                    current_value = txt_info[event_key][next_event_key]

                    # 检查 current_value 是否为数字或字符串
                    if isinstance(current_value, (int, float, str)):
                        try:
                            float_value = float(current_value)
                            txt_info[event_key][next_event_key] = {
                                'time': float_value,
                                'eeg_time': float(events_from_annot[i+1][0])
                            }
                        except ValueError:
                            print(f"Warning: Cannot convert {current_value} to float.")
                    else:
                        print(f"Warning: Value at txt_info[{event_key}][{next_event_key}] is not a number or string, it's a {type(current_value)}")
    time2t2t = {}
    for t1 in txt_info.keys():
        for t2 in txt_info[t1].keys():
            if type(txt_info[t1][t2]) == dict:
                time2t2t[txt_info[t1][t2]['time']] = [t1, t2]
    time_stamps = list(sorted([float(item) for item in time2t2t.keys()]))

    v2info = excel_info
    for v in v2info.keys():
        start_time = int(v2info[v]['start_time'] / 1e3)
        end_time = int(v2info[v]['end_time'] / 1e3)
        time_diff = int(end_time - start_time)
        eeg_time = find_time(start_time, time2t2t, time_stamps, txt_info)
        if eeg_time != None:
            v2info[v]['eeg_start_time'] = int(eeg_time)
        eeg_time = find_time(end_time, time2t2t, time_stamps, txt_info)
        if eeg_time != None:
            v2info[v]['eeg_end_time'] = int(eeg_time)
    return v2info

In [ ]:
, 'LAB2-guohongyang','LAB2-zhaochensong','LAB2-chenxingyu','LAB2-chenrong'

In [ ]:
for date in tqdm.tqdm([
           'LAB2-guohongyang','LAB2-zhaochensong','LAB2-chenxingyu','LAB2-chenrong'

]):
    student_id = date
    print(f'-----------------------------preprocessing data of user {student_id}-------------------------------')
    file_name = date + '.cnt'
        
    raw = mne.io.read_raw_cnt('./lab2-data/'+file_name, preload=True, verbose='WARNING')
    # tmp_file_name = '../data/eeg/' + str(student_id) + '_1 Data.cnt'
    # if os.path.exists(tmp_file_name):
    #     raw_0 = mne.io.read_raw_cnt(tmp_file_name, preload=True, verbose='WARNING')
    #     raw.append(raw_0)

    channels = ["FP1", "FPZ", "FP2", "AF3", "AF4", "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8", "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8", "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8", "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8", "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8", "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8", "CB1", "O1", "OZ", "O2", "CB2"]

    raw.pick_channels(channels)

    events_from_annot, event_dict = mne.events_from_annotations(raw, verbose='WARNING')
    #print(events_from_annot.shape)
    event2trigger_dic = dic_v2k(event_dict)
    for idx in range(len(events_from_annot)):
        events_from_annot[idx][2] = event2trigger_dic[events_from_annot[idx][2]]

    txt_info = load_txt('./lab2-txt/'+date+'.txt')
    #print(txt_info)
    #excel_info = load_excel('./lab1-log/'+date+'.xlsx')
    excel_info = load_excel_with_tend('./lab2-log/'+date+'.xlsx',user_data, renwu, pinyin_map)
    eeg_data = raw.get_data()
    events_from_annot = filter_255(events_from_annot)

    idx2eeg = {}
    idx = 0
    v2info = map_info(txt_info, events_from_annot, excel_info)

    for v in v2info.keys():
        v2info[v]['start_time'] = int(v2info[v]['start_time'])
        v2info[v]['end_time'] = int(v2info[v]['end_time'])
        if 'eeg_start_time' not in v2info[v].keys() or 'eeg_end_time' not in v2info[v].keys():
            continue
        if v2info[v]['eeg_end_time'] - v2info[v]['eeg_start_time'] < 0:
            print('error',v)
            continue
        time_diff = min(v2info[v]['eeg_end_time'] - v2info[v]['eeg_start_time'], 60 * 1000)
        idx2eeg[idx] = eeg_data[:,v2info[v]['eeg_start_time']:v2info[v]['eeg_start_time']+time_diff].tolist()
        v2info[v]['idx'] = int(idx)
        idx += 1

    json.dump(v2info, open('./v2info/'+date+'_v2info.json','w'))
    json.dump(idx2eeg, open('./x2eeg/'+date+'_idx2eeg.json','w'))

In [3]:
#显示得到的v2info的信息
def count_sorted_values_and_max_idx(json_data):
    tend_counts = {}
    video_type_counts = {}
    max_idx = None

    for entry in json_data.values():
        tend = entry.get('tend')
        video_type = entry.get('video_type')
        idx = entry.get('idx')

        if tend is not None:
            tend_counts[tend] = tend_counts.get(tend, 0) + 1

        if video_type is not None:
            video_type_counts[video_type] = video_type_counts.get(video_type, 0) + 1

        if idx is not None:
            if max_idx is None or idx > max_idx:
                max_idx = idx

    # Sorting the counts
    sorted_tend_counts = dict(sorted(tend_counts.items()))
    sorted_video_type_counts = dict(sorted(video_type_counts.items()))

    return sorted_tend_counts, sorted_video_type_counts, max_idx

In [8]:
def process_json_files_in_folder(folder_path):
    results = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                json_data = json.load(file)
                sorted_tend_counts, sorted_video_type_counts, max_idx = count_sorted_values_and_max_idx(json_data)
                results[filename] = {
                    "sorted_tend_counts": sorted_tend_counts,
                    "sorted_video_type_counts": sorted_video_type_counts,
                    "max_idx": max_idx
                }
    return results

# 使用示例（需要指定包含JSON文件的文件夹路径）
folder_path = './v2info/'
process_json_files_in_folder(folder_path)

{'LAB1-cangyueyang_v2info.json': {'sorted_tend_counts': {0: 2, 1: 18, 2: 21},
  'sorted_video_type_counts': {0: 10, 1: 10, 2: 5, 3: 5, 4: 5, 5: 5, 6: 1},
  'max_idx': 39},
 'LAB1-chenrong_v2info.json': {'sorted_tend_counts': {0: 10, 1: 10, 2: 20},
  'sorted_video_type_counts': {0: 10, 1: 10, 2: 5, 3: 5, 4: 5, 5: 5},
  'max_idx': 39},
 'LAB1-chenxingyu_v2info.json': {'sorted_tend_counts': {0: 4, 1: 16, 2: 21},
  'sorted_video_type_counts': {0: 10, 1: 10, 2: 5, 3: 5, 4: 5, 5: 5, 6: 1},
  'max_idx': 40},
 'LAB1-daisiwei_v2info.json': {'sorted_tend_counts': {0: 8, 1: 12, 2: 20},
  'sorted_video_type_counts': {0: 10, 1: 10, 2: 5, 3: 5, 4: 5, 5: 5},
  'max_idx': 39},
 'LAB1-dongyimeng_v2info.json': {'sorted_tend_counts': {0: 4, 1: 16, 2: 20},
  'sorted_video_type_counts': {0: 10, 1: 10, 2: 5, 3: 5, 4: 5, 5: 5},
  'max_idx': 39},
 'LAB1-fanhao_v2info.json': {'sorted_tend_counts': {0: 14, 1: 6, 2: 20},
  'sorted_video_type_counts': {0: 10, 1: 10, 2: 5, 3: 5, 4: 5, 5: 5},
  'max_idx': 38},
 'LA

In [9]:
def update_json_with_excel(json_data, excel_data, json_filename):
    """
    Update JSON data entries with 'valence', 'arousal', 'like', and normalized 'play_duration' values from Excel data.
    
    Parameters:
    json_data (dict): The JSON data to update.
    excel_data (pd.DataFrame): The Excel data as a Pandas DataFrame.
    json_filename (str): The JSON file name to match exactly with the member account in Excel data.
    
    Returns:
    dict: The updated JSON data.
    """
    
    def convert_time_to_normalized_value(time_val):
        """
        Convert time from datetime.time object to seconds and normalize.
        If time_val is NaN, return 0.
        """
        if pd.isna(time_val):
            return 0
        total_seconds = (time_val.hour * 3600 + time_val.minute * 60 + time_val.second) / 60
        return min(total_seconds, 1)

    # Iterate through each entry in the JSON data
    for entry_key in json_data.keys():
        # Extract the relevant rows from the Excel data
        relevant_rows = excel_data[excel_data['会员账号'].apply(lambda x: x in json_filename) & (excel_data['视频标题'] == entry_key)]

        # If relevant rows are found, update the JSON entry
        if not relevant_rows.empty:
            relevant_row = relevant_rows.iloc[0]  # Take the first match
            json_data[entry_key]['valence'] = relevant_row['valence']
            json_data[entry_key]['arousal'] = relevant_row['arousal']
            # Update 'like' based on whether the '点赞' column is marked with a "√"
            json_data[entry_key]['like'] = 1 if relevant_row['点赞'] == "√" else 0
            # Convert and normalize 'play_duration'
            play_duration = convert_time_to_normalized_value(relevant_row['观看时间'])
            json_data[entry_key]['play_duration'] = play_duration
            print(f"Updated {entry_key} with valence: {relevant_row['valence']}, arousal: {relevant_row['arousal']}, like: {json_data[entry_key]['like']}, and play_duration: {play_duration}")
        else:
            print(f"No match found for {entry_key} with member account: {json_filename}")
            
    return json_data


In [10]:
def update_json_files_in_folder(folder_path, excel_file_path):
    # Load the Excel data once, as it will be used for all JSON files
    excel_data = pd.read_excel(excel_file_path)

    # Iterate over all JSON files in the specified folder
    for json_file_path in glob.glob(os.path.join(folder_path, '*.json')):
        # Extract the json file name without extension to use in the update function
        json_filename = os.path.splitext(os.path.basename(json_file_path))[0]
        print("jsonname:",json_filename)

        # Read the current JSON file
        with open(json_file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)

        # Update the JSON data using the function provided
        updated_json_data = update_json_with_excel(json_data, excel_data, json_filename)

        # Write the updated JSON data back to the same file
        with open(json_file_path, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print(f"Updated file: {json_file_path}")

In [11]:
update_json_files_in_folder('./v2info', '评分总表2.xlsx')

jsonname: LAB1-cangyueyang_v2info
Updated LAB1-neutral-4 with valence: 6.0, arousal: 5.0, like: 0, and play_duration: 1.0
Updated LAB1-叶卡捷琳娜-正面 with valence: 7.0, arousal: 7.0, like: 1, and play_duration: 1
Updated LAB1-牛顿-正面 with valence: 4.0, arousal: 3.0, like: 0, and play_duration: 1.0
Updated LAB1-fear-4 with valence: 7.0, arousal: 9.0, like: 1, and play_duration: 1
Updated LAB1-项羽-负面 with valence: 3.0, arousal: 3.0, like: 0, and play_duration: 0.95
Updated LAB1-happy-4 with valence: 4.0, arousal: 5.0, like: 0, and play_duration: 1.0
Updated LAB1-fear-5 with valence: 8.0, arousal: 9.0, like: 1, and play_duration: 1
Updated LAB1-老白-正面 with valence: 6.0, arousal: 7.0, like: 1, and play_duration: 1.0
Updated LAB1-neutral-2 with valence: 5.0, arousal: 4.0, like: 0, and play_duration: 1.0
Updated LAB1-爱迪生-负面 with valence: 6.0, arousal: 6.0, like: 0, and play_duration: 1.0
Updated LAB1-fear-1 with valence: 7.0, arousal: 8.0, like: 1, and play_duration: 1
Updated LAB1-汉武帝-正面 with valence